<h5>Capstone Project: Business Problem</h5>
by Frantzdy Hervé
<h3>Introduction</h3> 
The car industry has been booming recently in the United States [Investopedia](https://www.investopedia.com/articles/pf/12/auto-industry.asp#:~:text=The%20U.S.%20economy%20was%20booming,million%20new%20cars%20were%20sold.&text=Among%20the%20more%20notable%20early,difficult%20recession%20of%202007%2D2008.). The tendency now favors more and more electric vehicles as they are viewed to be more environmentally-friendly and their design gives a feel of vehicles of the future. An automaker, namely Tesla, has particularly grown rapidly and its sales are projected to only increase in the future [History of the electric vehicles](https://www.energy.gov/articles/history-electric-car#:~:text=First%20Crude%20Electric%20Vehicle%20Is,an%20English%20inventor%20in%201884). The ambitious investor might also be wondering:'What is in it for me?'

If electric vehicles seem to be very attractive to the consumer, their price and the infrastructure that is required to facilitate their charging or even their construction do not seem to be particularly approachable. The latter seems to be the most challenging aspect of it all when one considers the fact that electric vehicles are almost completely different to the traditional vehicles when it comes to their engine outlook, their transmission system, their source of power, and the highly advanced technology needed for their computing wiring. In order to fathom how difficult these challenges are, we could preview what range of obstacles Tesla has had to face, (and continue to) just a few years ago to produce its cars from scratch in a gigafactory. Based on the poor financial performance they had just 3 years ago, there were some predictions indicating that they would not survive in the market for long. However, the persistence of its ceo has proved to pay off as he fafced all the challenges head on and planned accordingly. Today, this segment of the car industry in the US seems to be only promising and that might be what any investor willing to find a niche for highly profitable investments would be interested in.

<h3>Business Problem</h3>
As mentioned earlier, the electric vehicles represent a niche which requires massive investment to build the infrastructure needed. [Estimates for building a charging station](https://www.homeadvisor.com/cost/garages/install-an-electric-vehicle-charging-station/#:~:text=The%20national%20average%20for%20installing,median%20cost%20is%20%24731%20each). This obstacle could be largely offset, though, by the financial gain any smart investment in the electric car industry might yield. Based on the most pessimistic predictions, the traditional combustion engine will go extinct in about ten years while the increase in consumer's demand for electric cars and auto parts will continue at a staggering rate of 21% [Booming demand for electric vehicles](https://www.marketsandmarkets.com/Market-Reports/electric-vehicle-market-209371461.html). The key to grapple here is that the demand is not uniform in the US, all the states are not equal when it comes to growth in any sector. And, specifically, because of disparity in terms of the growth in the population by state and by urban area, job opportunities, and income per capita, one must be very cautious when targetting locations for their investment in said sector. As such, the need to identify the states and the cities where the demand for reliable transportation would be the most promising is then a good place to start. And this factor alone could be a good strategy to mobilize the funds needed as the early investors might break even from their investment in just a couple of quarters. However, the question remains, in what urban areas the demand for electric vehicles might be the highest? What states would then see these demands manifested the highest?

<h3>What we propose</h3>
The following lines will provide a comprehensive approach to predict the states where the demand for cars and auto parts will continue to grow and identify the urban areas where the population seems to be the most active. This solution is inspired by the data provided from the US Bureau of Economic Analysis (BEA) and the platform provided by Foursquare to locate areas where most venues are frequented. We will then leverage data using the BEA's API guidance and the well-structured platform designed by Foursquare. 
<dl>
  <dt>Our approach is two-fold:</dt>
  <dd>1. First, we pull data from the BEA's website related to the production level in each US state, the population, the income per capita and the consumer's spending for the past 11 years[^1]. This would allow us to use enough data that would increase the model accuracy in terms of predicting the 5 states with the highest demand for cars and auto parts in the years to come;</dd>
  <dd>2. Finally, we use the Foursquare platform to locate those urban areas where the population seems to be the most active from the states previously identified;</dd>
  This submission answers the second question of the assignment which is the presentation of the data section that we will use for the final project.
   </dl>
   
[^1]: The Consumer's consumption spending for cars and auto parts and the income per capita were not available for the year 2019.

<h3>Presentation of the scraping techniques used to leverage data from the BEA</h3>

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [2]:
#API request to the bureau of economic analysis's website
#Model for metadata request of list of available datasets:(http://apps.bea.gov/api/data?&UserID=Your-36CharacterKey&method=GETDATASETLIST&ResultFormat=JSON)
API_key='866C34B1-B3F1-4AB6-BD3A-C3F9D73FCC1D'
Method = 'GetData' #'GetParameterValues'
Result_format = 'JSON'
DataSetName ='NIUnderlyingDetail'
TableName ='U70205S' #reference for auto and Truck Unit Sales, Production, Inventories, Expenditures and Price
ParameterName = 'TableName'
Frequency = 'M'
Year_int = 2019
Year = str(Year_int)
GeoFips_C = 'COUNTY'
State = 'All'
Industry = 'All'

DN_Regional = 'Regional' # LineCode, TN, and GeoFIPS required
LineCode = 1 # or other digit?
TN_GDP_State = 'SQGDP9' # or 'SAGDP2S'
TN_Pop = 'SAINC51'
TN_consSpending = 'SAEXP2'
TN_IncPerCapita = 'CAINC1'
LC_GDP = 1

url_GDP = {'GDP_2019' : '', 'GDP_2018' : '', 'GDP_2017' : '', 'GDP_2016' : '', 'GDP_2015' : '', 'GDP_2014' : '', 'GDP_2013' : '', 'GDP_2012' : '',
'GDP_2011' : '', 'GDP_2010' : '', 'GDP_2009' : ''}

GDP_dict = {'GDP_2019' : '', 'GDP_2018' : '', 'GDP_2017' : '', 'GDP_2016' : '', 'GDP_2015' : '', 'GDP_2014' : '', 'GDP_2013' : '', 'GDP_2012' : '',
'GDP_2011' : '', 'GDP_2010' : '', 'GDP_2009' : ''}

GeoFips = 'STATE'

In [3]:
for i in range (0, 11):
    Year = str(Year_int - i)
    url_GDP[i] = 'http://apps.bea.gov/api/data?&UserID={}&method={}&DataSetName=Regional&TableName={}&LineCode={}&Year={}&GeoFips={}&ResultFormat={}'.format(
                  API_key, Method, TN_GDP_State, LC_GDP, Year, GeoFips, Result_format)
    GDP_dict[i] = requests.get(url_GDP[i]).json()
GDP_dict[3]

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'LINECODE',
     'ParameterValue': '1'},
    {'ParameterName': 'RESULTFORMAT', 'ParameterValue': 'JSON'},
    {'ParameterName': 'TABLENAME', 'ParameterValue': 'SQGDP9'},
    {'ParameterName': 'GEOFIPS', 'ParameterValue': 'STATE'},
    {'ParameterName': 'USERID',
     'ParameterValue': '866C34B1-B3F1-4AB6-BD3A-C3F9D73FCC1D'},
    {'ParameterName': 'METHOD', 'ParameterValue': 'GETDATA'},
    {'ParameterName': 'YEAR', 'ParameterValue': '2016'},
    {'ParameterName': 'DATASETNAME', 'ParameterValue': 'REGIONAL'}]},
  'Results': {'Statistic': 'Real GDP by state: All industry total',
   'UnitOfMeasure': 'Millions of chained 2012 dollars',
   'PublicTable': 'SQGDP9 All industry total',
   'UTCProductionTime': '2020-08-14T21:10:36.060',
   'NoteRef': ' ',
   'Dimensions': [{'Name': 'Code', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoFips', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoName', 'DataType': 'string', '

<h4> Data Wrangling to obtain csv files that contain the information needed for further analysis.</h4>

In [4]:
#Extracting the GDP data for 11 years
GDP_2019_0 = GDP_dict[0]
GDP_2019_0j = GDP_2019_0['BEAAPI']['Results']['Data']
GDP_2019_0df = json_normalize(GDP_2019_0j)
GDP_2019_0df.to_csv('GDP_2019_0df.csv')

GDP_2018_0 = GDP_dict[1]
GDP_2018_0j = GDP_2018_0['BEAAPI']['Results']['Data']
GDP_2018_0df = json_normalize(GDP_2018_0j)
GDP_2018_0df.to_csv('GDP_2018_0df.csv')

GDP_2017_0 = GDP_dict[2]
GDP_2017_0j = GDP_2017_0['BEAAPI']['Results']['Data']
GDP_2017_0df = json_normalize(GDP_2017_0j)
GDP_2017_0df.to_csv('GDP_2017_0df.csv')

GDP_2016_0 = GDP_dict[3]
GDP_2016_0j = GDP_2016_0['BEAAPI']['Results']['Data']
GDP_2016_0df = json_normalize(GDP_2016_0j)
GDP_2016_0df.to_csv('GDP_2016_0df.csv')

GDP_2015_0 = GDP_dict[4]
GDP_2015_0j = GDP_2015_0['BEAAPI']['Results']['Data']
GDP_2015_0df = json_normalize(GDP_2015_0j)
GDP_2015_0df.to_csv('GDP_2015_0df.csv')

GDP_2014_0 = GDP_dict[5]
GDP_2014_0j = GDP_2014_0['BEAAPI']['Results']['Data']
GDP_2014_0df = json_normalize(GDP_2014_0j)
GDP_2014_0df.to_csv('GDP_2014_0df.csv')

GDP_2013_0 = GDP_dict[6]
GDP_2013_0j = GDP_2013_0['BEAAPI']['Results']['Data']
GDP_2013_0df = json_normalize(GDP_2013_0j)
GDP_2013_0df.to_csv('GDP_2013_0df.csv')

GDP_2012_0 = GDP_dict[7]
GDP_2012_0j = GDP_2012_0['BEAAPI']['Results']['Data']
GDP_2012_0df = json_normalize(GDP_2012_0j)
GDP_2012_0df.to_csv('GDP_2012_0df.csv')

GDP_2011_0 = GDP_dict[8]
GDP_2011_0j = GDP_2011_0['BEAAPI']['Results']['Data']
GDP_2011_0df = json_normalize(GDP_2011_0j)
GDP_2011_0df.to_csv('GDP_2011_0df.csv')

GDP_2010_0 = GDP_dict[9]
GDP_2010_0j = GDP_2010_0['BEAAPI']['Results']['Data']
GDP_2010_0df = json_normalize(GDP_2010_0j)
GDP_2010_0df.to_csv('GDP_2010_0df.csv')

GDP_2009_0 = GDP_dict[10]
GDP_2009_0j = GDP_2009_0['BEAAPI']['Results']['Data']
GDP_2009_0df = json_normalize(GDP_2009_0j)
GDP_2009_0df.to_csv('GDP_2009_0df.csv')
GDP_2009_0df.head()

,Code,GeoFips,GeoName,TimePeriod,CL_UNIT,UNIT_MULT,DataValue
0,SQGDP9-1,00000,United States,2009Q1,Millions of chained 2012 dollars,6,"15,155,940.0"
1,SQGDP9-1,00000,United States,2009Q2,Millions of chained 2012 dollars,6,"15,134,117.0"
2,SQGDP9-1,00000,United States,2009Q3,Millions of chained 2012 dollars,6,"15,189,222.0"
3,SQGDP9-1,00000,United States,2009Q4,Millions of chained 2012 dollars,6,"15,356,058.0"
4,SQGDP9-1,01000,Alabama,2009Q1,Millions of chained 2012 dollars,6,"177,770.1"


In [5]:
url_pop = {'POP_2019' : '', 'POP_2018' : '', 'POP_2017' : '', 'POP_2016' : '', 'POP_2015' : '', 'POP_2014' : '', 'POP_2013' : '', 'GDP_2012' : '',
'POP_2011' : '', 'POP_2010' : '', 'POP_2009' : ''}

POP_dict = {'POP_2019' : '', 'POP_2018' : '', 'POP_2017' : '', 'POP_2016' : '', 'POP_2015' : '', 'POP_2014' : '', 'POP_2013' : '', 'GDP_2012' : '',
'POP_2011' : '', 'POP_2010' : '', 'POP_2009' : ''}

LC_Pop = 52 # Could add 51 too

In [6]:
for i in range (0, 11):
    Year = str(Year_int - i)
    url_pop[i] = 'http://apps.bea.gov/api/data?&UserID={}&method={}&DataSetName=Regional&TableName={}&LineCode={}&Year={}&GeoFips={}&ResultFormat={}'.format(
                  API_key, Method, TN_Pop, LC_Pop, Year, GeoFips, Result_format)
    POP_dict[i] = requests.get(url_pop[i]).json()
POP_dict[3]

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'TABLENAME',
     'ParameterValue': 'SAINC51'},
    {'ParameterName': 'RESULTFORMAT', 'ParameterValue': 'JSON'},
    {'ParameterName': 'METHOD', 'ParameterValue': 'GETDATA'},
    {'ParameterName': 'DATASETNAME', 'ParameterValue': 'REGIONAL'},
    {'ParameterName': 'LINECODE', 'ParameterValue': '52'},
    {'ParameterName': 'USERID',
     'ParameterValue': '866C34B1-B3F1-4AB6-BD3A-C3F9D73FCC1D'},
    {'ParameterName': 'GEOFIPS', 'ParameterValue': 'STATE'},
    {'ParameterName': 'YEAR', 'ParameterValue': '2016'}]},
  'Results': {'Statistic': 'Population',
   'UnitOfMeasure': 'Number of persons',
   'PublicTable': 'SAINC51 Population (persons)',
   'UTCProductionTime': '2020-08-14T21:10:45.560',
   'NoteRef': '1',
   'Dimensions': [{'Name': 'Code', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoFips', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoName', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'TimePer

In [7]:
POP_2019_0 = POP_dict[0]
POP_2019_0j = POP_2019_0['BEAAPI']['Results']['Data']
POP_2019_0df = json_normalize(POP_2019_0j)
POP_2019_0df.to_csv('POP_2019_0df.csv')

POP_2018_0 = POP_dict[1]
POP_2018_0j = POP_2018_0['BEAAPI']['Results']['Data']
POP_2018_0df = json_normalize(POP_2018_0j)
POP_2018_0df.to_csv('POP_2018_0df.csv')

POP_2017_0 = POP_dict[2]
POP_2017_0j = POP_2017_0['BEAAPI']['Results']['Data']
POP_2017_0df = json_normalize(POP_2017_0j)
POP_2017_0df.to_csv('POP_2017_0df.csv')

POP_2016_0 = POP_dict[3]
POP_2016_0j = POP_2016_0['BEAAPI']['Results']['Data']
POP_2016_0df = json_normalize(POP_2016_0j)
POP_2016_0df.to_csv('POP_2016_0df.csv')

POP_2015_0 = POP_dict[4]
POP_2015_0j = POP_2015_0['BEAAPI']['Results']['Data']
POP_2015_0df = json_normalize(POP_2015_0j)
POP_2015_0df.to_csv('POP_2015_0df.csv')

POP_2014_0 = POP_dict[5]
POP_2014_0j = POP_2014_0['BEAAPI']['Results']['Data']
POP_2014_0df = json_normalize(POP_2014_0j)
POP_2014_0df.to_csv('POP_2014_0df.csv')

POP_2013_0 = POP_dict[6]
POP_2013_0j = POP_2013_0['BEAAPI']['Results']['Data']
POP_2013_0df = json_normalize(POP_2013_0j)
POP_2013_0df.to_csv('POP_2013_0df.csv')

POP_2012_0 = POP_dict[7]
POP_2012_0j = POP_2012_0['BEAAPI']['Results']['Data']
POP_2012_0df = json_normalize(POP_2012_0j)
POP_2012_0df.to_csv('POP_2012_0df.csv')

POP_2011_0 = POP_dict[8]
POP_2011_0j = POP_2011_0['BEAAPI']['Results']['Data']
POP_2011_0df = json_normalize(POP_2011_0j)
POP_2011_0df.to_csv('POP_2011_0df.csv')

POP_2010_0 = POP_dict[9]
POP_2010_0j = POP_2010_0['BEAAPI']['Results']['Data']
POP_2010_0df = json_normalize(POP_2010_0j)
POP_2010_0df.to_csv('POP_2010_0df.csv')

POP_2009_0 = POP_dict[10]
POP_2009_0j = POP_2009_0['BEAAPI']['Results']['Data']
POP_2009_0df = json_normalize(POP_2009_0j)
POP_2009_0df.to_csv('POP_2009_0df.csv')
POP_2009_0df.head()

,Code,GeoFips,GeoName,TimePeriod,CL_UNIT,UNIT_MULT,DataValue,NoteRef
0,SAINC51-52,00000,United States *,2009,Number of persons,0,"306,771,529",*
1,SAINC51-52,01000,Alabama,2009,Number of persons,0,"4,757,938",NaN
2,SAINC51-52,02000,Alaska *,2009,Number of persons,0,"698,895",*
3,SAINC51-52,04000,Arizona,2009,Number of persons,0,"6,343,154",NaN
4,SAINC51-52,05000,Arkansas,2009,Number of persons,0,"2,896,843",NaN


In [8]:
url_cons = {'CON_2019' : '', 'CON_2018' : '', 'CON_2017' : '', 'CON_2016' : '', 'CON_2015' : '', 'CON_2014' : '', 'CON_2013' : '', 'GDP_2012' : '',
'CON_2011' : '', 'CON_2010' : '', 'CON_2009' : ''}

CONS_dict = {'CON_2019' : '', 'CON_2018' : '', 'CON_2017' : '', 'CON_2016' : '', 'CON_2015' : '', 'CON_2014' : '', 'CON_2013' : '', 'GDP_2012' : '',
'CON_2011' : '', 'CON_2010' : '', 'CON_2009' : ''}

LC_ConsSpend = 1 # motor vehicles and parts

In [9]:
for i in range (0, 11):
    Year = str(Year_int - i)
    url_cons[i] = 'http://apps.bea.gov/api/data?&UserID={}&method={}&DataSetName=Regional&TableName={}&LineCode={}&Year={}&GeoFips={}&ResultFormat={}'.format(
                  API_key, Method, TN_consSpending, LC_ConsSpend, Year, GeoFips, Result_format)
    CONS_dict[i] = requests.get(url_cons[i]).json()
CONS_dict[3]

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'GEOFIPS',
     'ParameterValue': 'STATE'},
    {'ParameterName': 'YEAR', 'ParameterValue': '2016'},
    {'ParameterName': 'USERID',
     'ParameterValue': '866C34B1-B3F1-4AB6-BD3A-C3F9D73FCC1D'},
    {'ParameterName': 'METHOD', 'ParameterValue': 'GETDATA'},
    {'ParameterName': 'TABLENAME', 'ParameterValue': 'SAEXP2'},
    {'ParameterName': 'DATASETNAME', 'ParameterValue': 'REGIONAL'},
    {'ParameterName': 'LINECODE', 'ParameterValue': '1'},
    {'ParameterName': 'RESULTFORMAT', 'ParameterValue': 'JSON'}]},
  'Results': {'Statistic': 'Per capita personal consumption expenditures (PCE) by state',
   'UnitOfMeasure': 'Dollars',
   'PublicTable': 'SAEXP2 Per capita personal consumption expenditures',
   'UTCProductionTime': '2020-08-14T21:10:53.200',
   'NoteRef': ' ',
   'Dimensions': [{'Name': 'Code', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoFips', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoName', 

In [10]:
CONS_2018_0 = CONS_dict[1]
CONS_2018_0j = CONS_2018_0['BEAAPI']['Results']['Data']
CONS_2018_0df = json_normalize(CONS_2018_0j)
CONS_2018_0df.to_csv('CONS_2018_0df.csv')

CONS_2017_0 = CONS_dict[2]
CONS_2017_0j = CONS_2017_0['BEAAPI']['Results']['Data']
CONS_2017_0df = json_normalize(CONS_2017_0j)
CONS_2017_0df.to_csv('CONS_2017_0df.csv')

CONS_2016_0 = CONS_dict[3]
CONS_2016_0j = CONS_2016_0['BEAAPI']['Results']['Data']
CONS_2016_0df = json_normalize(CONS_2016_0j)
CONS_2016_0df.to_csv('CONS_2016_0df.csv')

CONS_2015_0 = CONS_dict[4]
CONS_2015_0j = CONS_2015_0['BEAAPI']['Results']['Data']
CONS_2015_0df = json_normalize(CONS_2015_0j)
CONS_2015_0df.to_csv('CONS_2015_0df.csv')

CONS_2014_0 = CONS_dict[5]
CONS_2014_0j = CONS_2014_0['BEAAPI']['Results']['Data']
CONS_2014_0df = json_normalize(CONS_2014_0j)
CONS_2014_0df.to_csv('CONS_2014_0df.csv')

CONS_2013_0 = CONS_dict[6]
CONS_2013_0j = CONS_2013_0['BEAAPI']['Results']['Data']
CONS_2013_0df = json_normalize(CONS_2013_0j)
CONS_2013_0df.to_csv('CONS_2013_0df.csv')

CONS_2012_0 = CONS_dict[7]
CONS_2012_0j = CONS_2012_0['BEAAPI']['Results']['Data']
CONS_2012_0df = json_normalize(CONS_2012_0j)
CONS_2012_0df.to_csv('CONS_2012_0df.csv')

CONS_2011_0 = CONS_dict[8]
CONS_2011_0j = CONS_2011_0['BEAAPI']['Results']['Data']
CONS_2011_0df = json_normalize(CONS_2011_0j)
CONS_2011_0df.to_csv('CONS_2011_0df.csv')

CONS_2010_0 = CONS_dict[9]
CONS_2010_0j = CONS_2010_0['BEAAPI']['Results']['Data']
CONS_2010_0df = json_normalize(CONS_2010_0j)
CONS_2010_0df.to_csv('CONS_2010_0df.csv')

CONS_2009_0 = CONS_dict[10]
CONS_2009_0j = CONS_2009_0['BEAAPI']['Results']['Data']
CONS_2009_0df = json_normalize(CONS_2009_0j)
CONS_2009_0df.to_csv('CONS_2009_0df.csv')
CONS_2009_0df.head()

,Code,GeoFips,GeoName,TimePeriod,CL_UNIT,UNIT_MULT,DataValue
0,SAEXP2-1,00000,United States,2009,Dollars,0,"32,054"
1,SAEXP2-1,01000,Alabama,2009,Dollars,0,"26,587"
2,SAEXP2-1,02000,Alaska,2009,Dollars,0,"38,355"
3,SAEXP2-1,04000,Arizona,2009,Dollars,0,"28,596"
4,SAEXP2-1,05000,Arkansas,2009,Dollars,0,"25,678"


In [11]:
#We note that data for consumer's consumption spending for cars and auto parts was not available for the year 2019 on the site.

In [12]:
url_inc = {'INC_2019' : '', 'INC_2018' : '', 'INC_2017' : '', 'INC_2016' : '', 'INC_2015' : '', 'INC_2014' : '', 'INC_2013' : '', 'GDP_2012' : '',
'INC_2011' : '', 'INC_2010' : '', 'INC_2009' : ''}

INC_dict = {'INC_2019' : '', 'INC_2018' : '', 'INC_2017' : '', 'INC_2016' : '', 'INC_2015' : '', 'INC_2014' : '', 'INC_2013' : '', 'GDP_2012' : '',
'INC_2011' : '', 'INC_2010' : '', 'INC_2009' : ''}

LC_inc = 3

In [13]:
for i in range (0, 11):
    Year = str(Year_int - i)
    url_inc[i] = 'http://apps.bea.gov/api/data?&UserID={}&method={}&DataSetName=Regional&TableName={}&LineCode={}&Year={}&GeoFips={}&ResultFormat={}'.format(
                  API_key, Method, TN_IncPerCapita, LC_inc, Year, GeoFips, Result_format)
    INC_dict[i] = requests.get(url_cons[i]).json()
INC_dict[3]

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'YEAR',
     'ParameterValue': '2016'},
    {'ParameterName': 'METHOD', 'ParameterValue': 'GETDATA'},
    {'ParameterName': 'LINECODE', 'ParameterValue': '1'},
    {'ParameterName': 'USERID',
     'ParameterValue': '866C34B1-B3F1-4AB6-BD3A-C3F9D73FCC1D'},
    {'ParameterName': 'RESULTFORMAT', 'ParameterValue': 'JSON'},
    {'ParameterName': 'TABLENAME', 'ParameterValue': 'SAEXP2'},
    {'ParameterName': 'DATASETNAME', 'ParameterValue': 'REGIONAL'},
    {'ParameterName': 'GEOFIPS', 'ParameterValue': 'STATE'}]},
  'Results': {'Statistic': 'Per capita personal consumption expenditures (PCE) by state',
   'UnitOfMeasure': 'Dollars',
   'PublicTable': 'SAEXP2 Per capita personal consumption expenditures',
   'UTCProductionTime': '2020-08-14T21:11:00.870',
   'NoteRef': ' ',
   'Dimensions': [{'Name': 'Code', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoFips', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoName', 

In [14]:
INC_2018_0 = INC_dict[1]
INC_2018_0j = INC_2018_0['BEAAPI']['Results']['Data']
INC_2018_0df = json_normalize(INC_2018_0j)
INC_2018_0df.to_csv('INC_2018_0df.csv')

INC_2017_0 = INC_dict[2]
INC_2017_0j = INC_2017_0['BEAAPI']['Results']['Data']
INC_2017_0df = json_normalize(INC_2017_0j)
INC_2017_0df.to_csv('INC_2017_0df.csv')

INC_2016_0 = INC_dict[3]
INC_2016_0j = INC_2016_0['BEAAPI']['Results']['Data']
INC_2016_0df = json_normalize(INC_2016_0j)
INC_2016_0df.to_csv('INC_2016_0df.csv')

INC_2015_0 = INC_dict[4]
INC_2015_0j = INC_2015_0['BEAAPI']['Results']['Data']
INC_2015_0df = json_normalize(INC_2015_0j)
INC_2015_0df.to_csv('INC_2015_0df.csv')

INC_2014_0 = INC_dict[5]
INC_2014_0j = INC_2014_0['BEAAPI']['Results']['Data']
INC_2014_0df = json_normalize(INC_2014_0j)
INC_2014_0df.to_csv('INC_2014_0df.csv')

INC_2013_0 = INC_dict[6]
INC_2013_0j = INC_2013_0['BEAAPI']['Results']['Data']
INC_2013_0df = json_normalize(INC_2013_0j)
INC_2013_0df.to_csv('INC_2013_0df.csv')

INC_2012_0 = INC_dict[7]
INC_2012_0j = INC_2012_0['BEAAPI']['Results']['Data']
INC_2012_0df = json_normalize(INC_2012_0j)
INC_2012_0df.to_csv('INC_2012_0df.csv')

INC_2011_0 = INC_dict[8]
INC_2011_0j = INC_2011_0['BEAAPI']['Results']['Data']
INC_2011_0df = json_normalize(INC_2011_0j)
INC_2011_0df.to_csv('INC_2011_0df.csv')

INC_2010_0 = INC_dict[9]
INC_2010_0j = INC_2010_0['BEAAPI']['Results']['Data']
INC_2010_0df = json_normalize(INC_2010_0j)
INC_2010_0df.to_csv('INC_2010_0df.csv')

INC_2009_0 = INC_dict[10]
INC_2009_0j = INC_2009_0['BEAAPI']['Results']['Data']
INC_2009_0df = json_normalize(INC_2009_0j)
INC_2009_0df.to_csv('INC_2009_0df.csv')
INC_2009_0df.head()

,Code,GeoFips,GeoName,TimePeriod,CL_UNIT,UNIT_MULT,DataValue
0,SAEXP2-1,00000,United States,2009,Dollars,0,"32,054"
1,SAEXP2-1,01000,Alabama,2009,Dollars,0,"26,587"
2,SAEXP2-1,02000,Alaska,2009,Dollars,0,"38,355"
3,SAEXP2-1,04000,Arizona,2009,Dollars,0,"28,596"
4,SAEXP2-1,05000,Arkansas,2009,Dollars,0,"25,678"


In [15]:
#We note that data for consumer's consumption spending for cars and auto parts was not available for the year 2019 on the site.

<h3> Process of scraping Foursquare for specific locations within a major state</h3>
<dl>
  <dt>Important note:</dt>
    <dd> It is important to note that the data once obtained we will go ahead and analyze to discover any missing information or other formatting problems. Then we will try to see if there is any correlation between the different variables such as population growth and income per capita for instance. Once the data is ready, we will estimate a clustering model in order to predict the per capita consumer's spending for cars and auto parts. The model once built, we will then select the 5 states where the population growth is the highest. </dd>
    </dl>

In [16]:
CLIENT_ID = 'ZYOAVPSPIOIKRXIJGJ1EB02IKOQZW25JJ5JO04H1NSUAZM3Y' # your Foursquare ID
CLIENT_SECRET = 'UNLEX3EMVZDKCDXS5XGGT1I0MKRPKDHCU0NWUPVEBHAAVDUD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [17]:
#The city of Detroit is used here just as an illustration of how data would be extracted from Foursquare in further selecting major 
#urban areas

address = 'Detroit City, MI'

geolocator = Nominatim(user_agent="Detroit")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Detroit City, MI are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Detroit City, MI are 42.3315509, -83.0466403.


In [18]:
# create map of Detroit using latitude and longitude values for a clearer view of its main urban areas
map_Detroit = folium.Map(location=[latitude, longitude], zoom_start=11)
map_Detroit

In [19]:
# If Detroit is selected in our model, we will use the Foursquare app to leverage data for identifying its most populous 
#locations

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius in meters
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZYOAVPSPIOIKRXIJGJ1EB02IKOQZW25JJ5JO04H1NSUAZM3Y&client_secret=UNLEX3EMVZDKCDXS5XGGT1I0MKRPKDHCU0NWUPVEBHAAVDUD&v=20180605&ll=42.3315509,-83.0466403&radius=1000&limit=100'

In [20]:
results_detCity = requests.get(url).json()
results_detCity

{'meta': {'code': 200, 'requestId': '5f36feae86af52694591b3f0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Downtown Detroit',
  'headerFullLocation': 'Downtown Detroit, Detroit',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 90,
  'suggestedBounds': {'ne': {'lat': 42.340550909000015,
    'lng': -83.03448868842625},
   'sw': {'lat': 42.32255089099999, 'lng': -83.05879191157376}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '456fd59ff964a520333e1fe3',
       'name': 'Campus Martius',
       'location': {'address': 'Woodward Ave',
        'crossStreet': 'at Michigan Ave',
        'lat': 42.33157500935305,
        'lng

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
# Filtering the data to select only information of interest
venues_detCity = results_detCity['response']['groups'][0]['items']
    
nearby_venues_detCity = json_normalize(venues_detCity) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_detCity =nearby_venues_detCity.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_detCity['venue.categories'] = nearby_venues_detCity.apply(get_category_type, axis=1)

# clean columns
nearby_venues_detCity.columns = [col.split(".")[-1] for col in nearby_venues_detCity.columns]

nearby_venues_detCity.head()

,name,categories,lat,lng
0,Campus Martius,Park,42.331575,-83.046598
1,Avalon Cafe and Bakery,Café,42.332834,-83.047694
2,Texas de Brazil,Steakhouse,42.332293,-83.046711
3,Dime Store,American Restaurant,42.331039,-83.047734
4,Parc,American Restaurant,42.331564,-83.046700


In [23]:
#Calculation of the number of venues recorded on the platform
NumVenues_DetroitCity = nearby_venues_detCity.shape[0]
print('{} venues were returned by Foursquare.'.format(NumVenues_DetroitCity))

90 venues were returned by Foursquare.


<h5>Conclusion</h5>
From the comparison between several major agglomerations within a state selected from the model, we will then further select the three urban areas with the highest numbers of venues. This would be an indication of not only the region is populous but also that economic activities are increasing in said locations. This would then suggest that people would be very likely to buy a car or rely on public transportation systems to go about their activities.